# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [ ]:
import pandas as pd
import datetime
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

# Import own implementations
from milp import MILP
import genetic
from genetic import GA_Actions_Elite, GA_Actions_Tournament

# Importing tuning libraries
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

## Reading the Price data

In [ ]:
df = pd.read_csv("../01 - Data/example_week.csv")
df.head(2)

In [ ]:
df.info()

## The Power Plant

In [ ]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

## GA Actions

### Elite

In [ ]:
ga_solver = GA_Actions_Elite(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [ ]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": 200,
        "INITIAL_MUTATION_RATE": tune.choice(np.linspace(0.05, 0.5, 5)),
        "FINAL_MUTATION_RATE": tune.choice(np.linspace(0.01, 0.05, 5)),
        "INITIAL_EXPLORATION": 0.66,
        "ELITISM": tune.choice(np.linspace(0.05, 0.5, 10)),
    },
    total_generations=500,
    timeout_s=60*60*5,
)

In [ ]:
analysis.best_config

In [ ]:
pd.concat(analysis.trial_dataframes, axis=0).reset_index(drop=True).to_csv(f"./Tuning Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_GA_Elite.csv", index=False)

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
# (
#     analysis.dataframe()
#     .query("trial_id == 'd947e4c7'")
#     .filter(regex="config")
#     .melt()
#     .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
#     .set_index("variable")
#     .to_dict()["value"]
# )

### Tournament

In [ ]:
ga_solver = GA_Actions_Tournament(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [ ]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": 200,
        "CXPB": tune.uniform(0.05, 0.95),
        "INITIAL_MUTATION_RATE": np.linspace(0.05, 0.5, 5),
        "FINAL_MUTATION_RATE": np.linspace(0.01, 0.05, 5),
        "INITIAL_EXPLORATION": 0.66,
        "TOURNAMENT_SIZE": np.linspace(2, 100, 10),
    },
    total_generations=500,
    timeout_s=60*60*5,
)

In [ ]:
analysis.best_config

In [ ]:
pd.concat(analysis.trial_dataframes, axis=0).reset_index(drop=True).to_csv(f"./Tuning Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_GA_Tournament.csv", index=False)

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
(
    analysis.dataframe()
    .query("trial_id == 'dd531b71'")
    .filter(regex="config")
    .melt()
    .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
    .set_index("variable")
    .to_dict()["value"]
)